In [7]:
import sys
import os



import numpy as np
from SynthTempNetwork import Individual, SynthTempNetwork
from TemporalNetwork import ContTempNetwork, StaticTempNetwork
from FlowStability import FlowIntegralClustering, run_multi_louvain
import parallel_clustering
import pickle

import matplotlib.pyplot as plt
import matplotlib

from scipy.sparse import (lil_matrix, dok_matrix, diags, eye, isspmatrix_csr, isspmatrix,
                          csr_matrix, coo_matrix, csc_matrix)
from scipy.sparse.linalg import eigsh

In [2]:
lamdas=np.logspace(-4,2,20)
print(lamdas)
between_lamdas=np.linspace(-4,2,20)

[1.00000000e-04 2.06913808e-04 4.28133240e-04 8.85866790e-04
 1.83298071e-03 3.79269019e-03 7.84759970e-03 1.62377674e-02
 3.35981829e-02 6.95192796e-02 1.43844989e-01 2.97635144e-01
 6.15848211e-01 1.27427499e+00 2.63665090e+00 5.45559478e+00
 1.12883789e+01 2.33572147e+01 4.83293024e+01 1.00000000e+02]


In [3]:
net = ContTempNetwork.load('fig3_network_short')

In [4]:
list_forw_flows = []

for i in lamdas:
    list_forw_flows.append(FlowIntegralClustering(T_inter_list=net.inter_T[i],
                                  time_list=net.times,
                                  verbose=True))

PID  55705  : computing T_list
PID  55705  : computing integral
PID  55705  :  0  over  459  took 0.00s
integral took 0.13s
PID  55705  : computing T_list
PID  55705  : computing integral
PID  55705  :  0  over  459  took 0.00s
integral took 0.13s
PID  55705  : computing T_list
PID  55705  : computing integral
PID  55705  :  0  over  459  took 0.00s
integral took 0.12s
PID  55705  : computing T_list
PID  55705  : computing integral
PID  55705  :  0  over  459  took 0.00s
integral took 0.12s
PID  55705  : computing T_list
PID  55705  : computing integral
PID  55705  :  0  over  459  took 0.00s
integral took 0.13s
PID  55705  : computing T_list
PID  55705  : computing integral
PID  55705  :  0  over  459  took 0.00s
integral took 0.13s
PID  55705  : computing T_list
PID  55705  : computing integral
PID  55705  :  0  over  459  took 0.00s
integral took 0.13s
PID  55705  : computing T_list
PID  55705  : computing integral
PID  55705  :  0  over  459  took 0.00s
integral took 0.13s
PID  557

In [5]:
flowintegralclustering = list_forw_flows[0]

In [6]:
clusters, stabilites, seeds = parallel_clustering.compute_parallel_clustering(flowintegralclustering, num_repeat=3, nproc=3, 
                                verbose=True, n_meta_iter_max=1000, 
                                n_sub_iter_max=1000, 
                                clust_verbose=False, print_num_loops=False)

AttributeError: 'FlowIntegralClustering' object has no attribute 'p2'

In [8]:
run_multi_louvain(clustering=flowintegralclustering, num_repeat=3)

AttributeError: 'FlowIntegralClustering' object has no attribute 'p2'

In [ ]:
for i in range(len(lamdas)):
    list_forw_flows[i].find_louvain_clustering()

In [ ]:
list_forw_flows[18].clustering[0]

### Generating/Saving/Loading static networks

In [ ]:
# #Generating Networks

static_net_adj = net.compute_static_adjacency_matrix()

#weighted
#Combinatorial Laplacian
static_net_weighted_heat = StaticTempNetwork(times = net.times, adjacency = static_net_adj)
static_net_weighted_heat.compute_laplacian_matrices(random_walk = False)
# for i in lamdas:
#     static_net_weighted_heat.compute_inter_transition_matrices(lamda=i, dense_expm=False, use_sparse_stoch=False)
#     static_net_weighted_heat.compute_transition_matrices(lamda=i)

#Random Walk Laplacian
static_net_weighted = StaticTempNetwork(times = net.times, adjacency = static_net_adj)
static_net_weighted.compute_laplacian_matrices(random_walk = True)
# for i in lamdas:
#     static_net_weighted.compute_inter_transition_matrices(lamda=i, dense_expm=False, use_sparse_stoch=False)
#     static_net_weighted.compute_transition_matrices(lamda=i)


#unweighted
#entries are sum of durations of events, need to set to 1
static_net_adj = static_net_adj.toarray()
static_net_adj[static_net_adj != 0] = 1
static_net_adj = csr_matrix(static_net_adj)

#Combinatorial Laplacian
static_net_heat = StaticTempNetwork(times = net.times, adjacency = static_net_adj)
static_net_heat.compute_laplacian_matrices(random_walk = False)
# for i in lamdas:
#     static_net_heat.compute_inter_transition_matrices(lamda=i, dense_expm=False, use_sparse_stoch=False)
#     static_net_heat.compute_transition_matrices(lamda=i)

#Random Walk Laplacian
static_net = StaticTempNetwork(times = net.times, adjacency = static_net_adj)
static_net.compute_laplacian_matrices(random_walk = True)
# for i in lamdas:
#     static_net.compute_inter_transition_matrices(lamda=i, dense_expm=False, use_sparse_stoch=False)
#     static_net.compute_transition_matrices(lamda=i)

In [ ]:
#Saving Networks
static_net_weighted_heat.save('fig3_network_static_weighted_heat')
static_net_weighted.save('fig3_network_static_weighted')
static_net_heat.save('fig3_network_static_heat')
static_net.save('fig3_network_static')

In [ ]:
# #Loading Networks
# static_net_weighted_heat = StaticTempNetwork.load('fig3_network_static_weighted_heat')
# static_net_weighted = StaticTempNetwork.load('fig3_network_static_weighted')
# static_net_heat = StaticTempNetwork.load('fig3_network_static_heat')
# static_net = StaticTempNetwork.load('fig3_network_static')

In [ ]:
def one_time(s):
    one_time_s = []
    for key in s.keys():
        for i in s[key]:
            one_time_s.append(i)
    return one_time_s

def derivative(v1, v2):
    assert len(v1) == len(v2), "Error: in order to compute the numerical derivative, the two varibales need to have the same lenght."
    derivative = []
    for i, vi in enumerate(v1[1:]):
        m1 =(vi - v1[i-1])/(v2[i]-v2[i-1])
        m2 =(v1[i+1] - vi)/(v2[i+1]-v2[i])
        derivative.append((m1+m2)/2)
    return derivative

# Unweighted Static Network

### Spectral vNS

In [ ]:
for i in lamdas:
    static_net_heat.compute_spectral_vonNeumann_entropy(lamda=i, force_csr=True, time_domain = static_net_heat.times[-1:])

In [ ]:
plt.subplot(1, 2, 1)
plt.plot(lamdas, one_time(static_net_heat.spectral_vNS))

plt.subplot(1, 2, 2)
plt.plot(lamdas, one_time(static_net_heat.spectral_vNS))
plt.yscale('log')
plt.title('Spectral vNS')

### Von Neumann Entropy

In [ ]:
for i in lamdas:
    static_net_heat.compute_vonNeumann_entropy(lamda=i, force_csr=True, time_domain = static_net_heat.times[-1:])

In [ ]:
plt.subplot(1, 2, 1)
plt.plot(lamdas, one_time(static_net_heat.vNS))

plt.subplot(1, 2, 2)
plt.plot(lamdas, one_time(static_net_heat.vNS))
plt.yscale('log')
plt.title('vNS')

In [ ]:
#comparison between the two

print(one_time(static_net_heat.vNS))
print(one_time(static_net_heat.spectral_vNS))

## Derivatives

In [ ]:
St = []
for j, lamda in enumerate(lamdas):
    St = one_time(static_net_heat.vNS)

v1 = St
v2 = between_lamdas
ds_dt = derivative(v1, v2)

plt.subplot(1, 2, 1)
plt.plot(lamdas[1:], ds_dt, label='t = ')

plt.subplot(1, 2, 2)
plt.plot(lamdas[1:], ds_dt, label='t = ')
plt.xscale('log')

plt.legend(fontsize='xx-small', loc='upper left', bbox_to_anchor=(1.05, 1))
plt.title('Static Network: d(Spectral vNS) / dlamda')
#plt.savefig('./figures/fig3_recap_entropy_plots/fig3_dspectral_vNS_dlamda.eps', format='eps')

plt.show()

# Weighted Static network

In [ ]:
for i in lamdas:
    static_net_weighted_heat.compute_spectral_vonNeumann_entropy(lamda=i, force_csr=True, time_domain = static_net_weighted_heat.times[-1:])

In [ ]:
for i in lamdas:
    static_net_weighted_heat.compute_vonNeumann_entropy(lamda=i, force_csr=True, time_domain = static_net_weighted_heat.times[-1:])

In [ ]:
#comparison between the two

print(one_time(static_net_weighted_heat.vNS))
print(one_time(static_net_weighted_heat.spectral_vNS))

In [ ]:
plt.subplot(1, 2, 1)
plt.plot(lamdas, one_time(static_net_weighted_heat.vNS))

plt.subplot(1, 2, 2)
plt.plot(lamdas, one_time(static_net_weighted_heat.vNS))
plt.yscale('log')
plt.title('vNS')

In [ ]:
for j, lamda in enumerate(lamdas):
    St = one_time(static_net_weighted_heat.vNS)

    v1 = St
    v2 = between_lamdas
    ds_dt = derivative(v1, v2)

plt.subplot(1, 2, 1)
plt.plot(lamdas[1:], ds_dt, label='t = ')

plt.subplot(1, 2, 2)
plt.plot(lamdas[1:], ds_dt, label='t = ')
plt.xscale('log')

plt.legend(fontsize='xx-small', loc='upper left', bbox_to_anchor=(1.05, 1))
plt.title('Static Network: d(vNS) / dlamda')
#plt.savefig('./figures/fig3_recap_entropy_plots/fig3_dvNS_dlamda.eps', format='eps')

plt.show()

# Static Network Random Walk 

In [ ]:
for i in lamdas:
    static_net.compute_vonNeumann_entropy(lamda=i, force_csr=True, time_domain = static_net_heat.times[-1:])

In [ ]:
plt.subplot(1, 2, 1)
plt.plot(lamdas, one_time(static_net.vNS))

plt.subplot(1, 2, 2)
plt.plot(lamdas, one_time(static_net.vNS))
plt.yscale('log')
plt.title('vNS')

In [ ]:
St = []
for j, lamda in enumerate(lamdas):
    St = one_time(static_net.vNS)

v1 = St
v2 = between_lamdas
ds_dt = derivative(v1, v2)

plt.subplot(1, 2, 1)
plt.plot(lamdas[1:], ds_dt, label='t = ')

plt.subplot(1, 2, 2)
plt.plot(lamdas[1:], ds_dt, label='t = ')
plt.xscale('log')

plt.legend(fontsize='xx-small', loc='upper left', bbox_to_anchor=(1.05, 1))
plt.title('Static Network: d(vNS) / dlamda')
#plt.savefig('./figures/fig3_recap_entropy_plots/fig3_dspectral_vNS_dlamda.eps', format='eps')

plt.show()

# Weighted Static Network Random Walk

In [ ]:
for i in lamdas:
    static_net_weighted.compute_vonNeumann_entropy(lamda=i, force_csr=True, time_domain = static_net_heat.times[-1:])

In [ ]:
plt.subplot(1, 2, 1)
plt.plot(lamdas, one_time(static_net_weighted.vNS))

plt.subplot(1, 2, 2)
plt.plot(lamdas, one_time(static_net_weighted.vNS))
plt.yscale('log')
plt.title('vNS')

In [ ]:
St = []
for j, lamda in enumerate(lamdas):
    St = one_time(static_net_weighted.vNS)

v1 = St
v2 = between_lamdas
ds_dt = derivative(v1, v2)

plt.subplot(1, 2, 1)
plt.plot(lamdas[1:], ds_dt, label='t = ')

plt.subplot(1, 2, 2)
plt.plot(lamdas[1:], ds_dt, label='t = ')
plt.xscale('log')

plt.legend(fontsize='xx-small', loc='upper left', bbox_to_anchor=(1.05, 1))
plt.title('Static Network: d(vNS) / dlamda')
#plt.savefig('./figures/fig3_recap_entropy_plots/fig3_dspectral_vNS_dlamda.eps', format='eps')

plt.show()

In [ ]:
with open('test_lamda100.000000', 'rb') as f:
    pickle_test = pickle.load(f)

In [ ]:
pickle_test.keys()

In [ ]:
len(pickle_test['vNS'])

In [ ]:
pickle_test['vNS']

In [ ]:
tuple([round(lamda, 6) for lamda in lamdas ])

In [ ]:
file='fig3_lamdas'
lamdasf6= [round(lamda, 6) for lamda in lamdas ]
with open(file, "w") as output:
    output.write('(')
    for lamda in lamdasf6[:-1]:
        output.write(str(lamda) + ' ' )
    output.write(str(lamdasf6[-1]) + ')')